In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
path = 'lista_tfm.csv'

In [4]:
df = pd.read_csv(path , sep = ',' ,  error_bad_lines=False ,index_col=0)

# DATA CLEANING

## Creating a new column for the latest year available

In [5]:
df['Ultimo año disponible'] = pd.to_datetime(df['Ultimo año disponible'])
df['Ultimo año disponible'] = pd.DatetimeIndex(df['Ultimo año disponible']).year

## Starting the EDA: finding wrong coordinates

In [6]:
columns_wrong_coordinates = list(df[(df['Coordenada - X'] == 0.0) | (df['Coordenada - Y']== 0.0)].index)

In [7]:
df.loc[columns_wrong_coordinates]

,Nombre,Código NIF,Localidad,Coordenada - X,Coordenada - Y,Ultimo año disponible,Ultimo número empleados,No of companies in corporate group,Código primario CNAE 2009,Literal código CNAE 2009 primario,Descripción actividad,Ingresos de explotación EUR Últ. año disp.,Ingresos de explotación EUR 2020,Ingresos de explotación EUR 2019,Ingresos de explotación EUR 2018,Ingresos de explotación EUR 2017,Ingresos de explotación EUR 2016,Ingresos de explotación EUR 2015,Ingresos de explotación EUR 2014,Ingresos de explotación EUR 2013,Ingresos de explotación EUR 2012,Ingresos de explotación EUR 2011,Ingresos de explotación EUR 2010,Resultado del Ejercicio EUR Últ. año disp.,Resultado del Ejercicio EUR 2020,Resultado del Ejercicio EUR 2019,Resultado del Ejercicio EUR 2018,Resultado del Ejercicio EUR 2017,Resultado del Ejercicio EUR 2016,Resultado del Ejercicio EUR 2015,Resultado del Ejercicio EUR 2014,Resultado del Ejercicio EUR 2013,Resultado del Ejercicio EUR 2012,Resultado del Ejercicio EUR 2011,Resultado del Ejercicio EUR 2010,Rentabilidad económica (%) % Últ. año disp.,Rentabilidad económica (%) % 2020,Rentabilidad económica (%) % 2019,Rentabilidad económica (%) % 2018,Rentabilidad económica (%) % 2017,Rentabilidad económica (%) % 2016,Rentabilidad económica (%) % 2015,Rentabilidad económica (%) % 2014,Rentabilidad económica (%) % 2013,Rentabilidad económica (%) % 2012,Rentabilidad económica (%) % 2011,Rentabilidad económica (%) % 2010,Rentabilidad financiera (%) % Últ. año disp.,Rentabilidad financiera (%) % 2020,Rentabilidad financiera (%) % 2019,Rentabilidad financiera (%) % 2018,Rentabilidad financiera (%) % 2017,Rentabilidad financiera (%) % 2016,Rentabilidad financiera (%) % 2015,Rentabilidad financiera (%) % 2014,Rentabilidad financiera (%) % 2013,Rentabilidad financiera (%) % 2012,Rentabilidad financiera (%) % 2011,Rentabilidad financiera (%) % 2010,Liquidez general % Últ. año disp.,Liquidez general % 2020,Liquidez general % 2019,Liquidez general % 2018,Liquidez general % 2017,Liquidez general % 2016,Liquidez general % 2015,Liquidez general % 2014,Liquidez general % 2013,Liquidez general % 2012,Liquidez general % 2011,Liquidez general % 2010,Endeudamiento (%) % Últ. año disp.,Endeudamiento (%) % 2020,Endeudamiento (%) % 2019,Endeudamiento (%) % 2018,Endeudamiento (%) % 2017,Endeudamiento (%) % 2016,Endeudamiento (%) % 2015,Endeudamiento (%) % 2014,Endeudamiento (%) % 2013,Endeudamiento (%) % 2012,Endeudamiento (%) % 2011,Endeudamiento (%) % 2010,Número empleados Últ. año disp.,Número empleados 2020,Número empleados 2019,Número empleados 2018,Número empleados 2017,Número empleados 2016,Número empleados 2015,Número empleados 2014,Número empleados 2013,Número empleados 2012,Número empleados 2011,Número empleados 2010,Distribución - Mujeres Últ. año disp.,Distribución - Mujeres 2020,Distribución - Mujeres 2019,Distribución - Mujeres 2018,Distribución - Mujeres 2017,Distribución - Mujeres 2016,Distribución - Mujeres 2015,Distribución - Mujeres 2014,Distribución - Mujeres 2013,Distribución - Mujeres 2012,Distribución - Mujeres 2011,Distribución - Mujeres 2010,Distribución - Hombres Últ. año disp.,Distribución - Hombres 2020,Distribución - Hombres 2019,Distribución - Hombres 2018,Distribución - Hombres 2017,Distribución - Hombres 2016,Distribución - Hombres 2015,Distribución - Hombres 2014,Distribución - Hombres 2013,Distribución - Hombres 2012,Distribución - Hombres 2011,Distribución - Hombres 2010
140.0,INFORTEC CONSULTORES SA,A41704651,SEVILLA,0.0,0.0,2019,500,8.0,6202,Actividades de consultoría informática,Servicios de consultoría informática.,"38,546,776",n.d.,"38,546,776","36,170,932","31,644,378","36,293,281","39,827,577","35,222,916","39,636,213","36,370,523","31,095,475","29,885,692","639,659",n.d.,"639,659","542,335","579,860","198,290","596,404","342,145","259,170","405,763","378,855","228,744",9.15,n.d.,9.15,8.48,9.91,3.54,13.39,9.41,8.24,7.67,10.30,6.44,19.88,n.d.,19.88,19.47,24.69,9.37,35.26,27.37,22.41,41.43,55.18,54.0

In [8]:
df = df.drop(columns_wrong_coordinates, axis = 0).reset_index(drop=True)

###  *TO BE DONE : These coordinates should be input by hand*

## Change column names

In [9]:
columns_clean = []
for column in df.columns:
    if column == str('Código NIF') :   column = 'NIF'
    if column == str('Código primario CNAE 2009') :   column = 'CNAE2009'  
    if column == str('Literal código CNAE 2009 primario') :   column = 'CNAE2009Descr'   
    columns_clean.append(column)

columns_clean

df_info = df
df_info.columns = columns_clean

In [10]:
info_columns = ['Nombre' , 'NIF' , 'Localidad' , 'Coordenada - X' , 'Coordenada - Y' , 
                'No of companies in corporate group' , 'CNAE2009',  'CNAE2009Descr' , 'Descripción actividad' ]
df_info = df_info[info_columns]

## Create financial dataset
### In years

In [11]:
df_years = pd.DataFrame()
df_financial = pd.DataFrame()

In [12]:
initial_year = 2010
end_year = 2020
for año in range(initial_year, end_year):
  df_years= df[df.columns[df.columns.str.contains("Nombre") | df.columns.str.contains("\n" + str(año))]]
  df_years.insert(1 , 'Año' , año)
  df_years.columns = df_years.columns.str.replace('\nEUR'  , ''  ).str.replace('\n%'  , '' ).str.replace("\n" + str(año)  , ''  )
  df_financial = df_financial.append(df_years ,ignore_index= True )
#df_years['Año'] = 2017

## Removing the commas and the n.d.

#### Selecting everything except the column called Nombre

In [13]:
df_financial.loc[:, df_financial.columns != 'Nombre'] = df_financial.loc[:, df_financial.columns != 'Nombre'].replace(',','', regex=True)
df_financial.loc[:, df_financial.columns != 'Nombre'] = df_financial.loc[:, df_financial.columns != 'Nombre'].replace('n.d.',0, regex=True)
df_financial.loc[:, df_financial.columns != 'Nombre'] = df_financial.loc[:, df_financial.columns != 'Nombre'].replace('n.s.',0, regex=True)

In [14]:
df_financial.loc[:, df_financial.columns != 'Nombre'] = df_financial.loc[:, df_financial.columns != 'Nombre'].apply(pd.to_numeric)

### Create a joint df for ploting

In [15]:
df_join= pd.merge(df_info, df_financial, how = 'inner' , on = 'Nombre')

In [16]:
df_join

,Nombre,NIF,Localidad,Coordenada - X,Coordenada - Y,No of companies in corporate group,CNAE2009,CNAE2009Descr,Descripción actividad,Año,Ingresos de explotación,Resultado del Ejercicio,Rentabilidad económica (%),Rentabilidad financiera (%),Liquidez general,Endeudamiento (%),Número empleados,Distribución - Mujeres,Distribución - Hombres
0,AMADEUS IT GROUP SA,A84236934,MADRID,-3.657597,40.436671,176.0,6311,"Proceso de datos, hosting y actividades relaci...","Servicios de procesamiento de datos,.",2010,530402709,463892670,83.16,101.29,1.70,17.90,4,0,0
1,AMADEUS IT GROUP SA,A84236934,MADRID,-3.657597,40.436671,176.0,6311,"Proceso de datos, hosting y actividades relaci...","Servicios de procesamiento de datos,.",2011,346773540,324021740,21.52,58.62,1.21,63.29,16,2,14
2,AMADEUS IT GROUP SA,A84236934,MADRID,-3.657597,40.436671,176.0,6311,"Proceso de datos, hosting y actividades relaci...","Servicios de procesamiento de datos,.",2012,272501000,228655000,15.90,41.10,1.36,61.32,15,2,13
3,AMADEUS IT GROUP SA,A84236934,MADRID,-3.657597,40.436671,176.0,6311,"Proceso de datos, hosting y actividades relaci...","Servicios de procesamiento de datos,.",2013,323074360,280287030,18.19,46.04,1.55,60.49,4,1,3
4,AMADEUS IT GROUP SA,A84236934,MADRID,-3.657597,40.436671,176.0,6311,"Proceso de datos, hosting y actividades relaci...","Servicios de procesamiento de datos,.",2014,368788000,324911000,20.55,93.78,0.66,78.09,4,3,1
5,AMADEUS IT GROUP SA,A84236934,MADRID,-3.657597,40.436671,176.0,6311,"Proceso de datos, hosting y actividades relaci...","Servicios de procesamiento de datos,.",2015,402469000,344846000,24.00,91.83,0.88,73.86,14,2,12
6,AMADEUS IT GROUP SA,A84236934,MADRID,-3.657597,40.436671,176.0,6311,"Proceso de datos, hosting y actividades relaci...","Servicios de procesamiento de datos,.",2016,3973100000,701100000,14.91,57.58,0.43,74.11,847,477,444
7,AMADEUS IT GROUP SA,A84236934,MADRID,-3.657597,40.436671,176.0,6311,"Proceso de datos, hosting y actividades relaci...","Servicios de procesamiento de datos,.",2017,4308800000,596100000,11.95,58.24,0.48,79.47,971,539,489
8,AMADEUS IT GROUP SA,A84236934,MADRID,-3.657597,40.436671,176.0,6311,"Proceso de datos, hosting y actividades relaci...","Servicios de procesamiento de datos,.",2018,4520600000,608400000,11.17,63.16,0.51,82.32,976,532,473
9,AMADEUS IT GROUP SA,A84236934,MADRID,-3.657597,40.436671,176.0,6311,"Proceso de datos, hosting y actividades relaci...","Servicios de procesamiento de datos,.",2019,4744600000,643400000,11.86,58.58,0.33,79.75,1012,548,475


# PLOTLY: course


In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import plotly.express as px
import dash_table as dt
import json

In [ ]:
mapbox_access_token = open("AlvaroMapBoxToken.mapbox_token").read()

#### Creating dropdown for:
- CNAE 2009 type
- Year

In [ ]:
type_dropdown = []

for type_company in df_info['CNAE2009'].unique():
    description_type_company = df_info[df_info['CNAE2009'] == type_company]['CNAE2009Descr'].values[0]
    type_dropdown.append({'label': str(description_type_company) , 'value' : str(type_company) })


In [ ]:
year_dropdown = []

for year_company in df_financial['Año'].unique():
    year_dropdown.append({'label': str(year_company) , 'value' : str(year_company) })

### Starting Dash

In [ ]:
app = dash.Dash()

In [ ]:
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

Markdown text

In [ ]:
markdown_text = '''
### How to use this interactive graph

*This map has been built using data from [Sabi Database](https://sabi.bvdinfo.com/version-202115/home.serv?product=SabiNeo)*

Filter by: 


- Type of organization
- Year

Then click in **Update** buttom.

If you want to see more info about any company, click on it


'''

In [ ]:
app.layout = html.Div(children = [
        
        #HEADER   
        html.H1(
        children='Brújula para Data Scientist',
        style={
            'textAlign': 'center',
            'color': colors['text']
            }
        ),
    
        #Introduction text
        html.Div(
        children='In this iteractive graph you will be able to find the companies you always desired',
        style={
            'textAlign': 'center',
            'color': colors['text']
            }
        ),
    
    
        # MARKDOWN 
        dcc.Markdown(children=markdown_text),
    
    
        #DROPDOWN COMPANY TYPE
        html.Div([
            dcc.Dropdown(
                id='type-dropdown',
                options=type_dropdown,
                value=['6202', '6311'], #Initial values
                multi=True
            )
        ],
        style={'width': '48%', 'display': 'inline-block'}),
    
        #DROPDOWN YEAR 
        html.Div([
            dcc.Dropdown(
                id='year-dropdown',
                options=year_dropdown,
                value= str(2010)
            )
        ],
        style={'width': '48%', 'display': 'inline-block'}),
    
    
    

        # SLIDER https://dash.plot.ly/dash-core-components/slider
        html.P(
            dcc.Slider(
                id='slide-revenue',
                min=0,
                max=500000000,
                step=None,
                marks=({int(i*(500000000/10)): str(int(i*(500000000/10))) + ' €' for i in range(0,11)}),  #Review why is not working
                value=0
            ),
                style={
                    'color': '#050000'
                    }
        ),
    
        #TEST
        html.Div(
            id='test-div',
            style={
                'textAlign': 'center',
                'color': colors['text'],
                'width': '48%', 
                'display': 'inline-block'
            }
        ),
    
    
        # UPDATE BUTTON 
        html.Button(
            id='update-button',
            n_clicks=0,
            children='Update graph',
            style={
                'fontSize':28,
                'textAlign': 'center',
                'color': colors['text'],
                'width': '20%', 
                'display': 'inline-block'
                  }
        ),
    
        # RADIO ITEMS https://dash.plot.ly/dash-core-components/radioitems
        #html.Label('Radio Items'),
        #dcc.RadioItems(
        #    options=[
        #    {'label': 'New York City', 'value': 'NYC'},
        #    {'label': 'Montréal', 'value': 'MTL'},
        #    {'label': 'San Francisco', 'value': 'SF'}
        #    ],
        #    value='MTL'
        #)],
                      
        #style={'backgroundColor': colors['background']}
        #),
    
        # GRAPH 
        html.Div([
            dcc.Loading(dcc.Graph(
                    id='map-companies')
                    , style={'verticalAlign':'middle','height' : '1200px', 'paddingTop' : '175px'})
            ]),
    

    
        # TABLE 
        dt.DataTable(
                    id='hover-data',
                    style_data={
                        'whiteSpace': 'normal',
                        'height': 'auto'
                        },
                     style_table={
                        'maxHeight': '700px'
                        ,'overflowY': 'scroll'
                        },
            
                    style_cell_conditional=[
                        {'if': {'column_id': 'Nombre'},
                                 'width': '125px'},
                        {'if': {'column_id': 'CNAE2009Descr'},
                                 'width': '450px'
                                 ,'height':'auto'},
                                ],
                    style_cell={
                        'overflow': 'hidden',
                        'textOverflow': 'ellipsis',
                        'maxWidth': '75px'
                            }
                    )
    
    
])

With State I could wait till a button is clicked for example

In [ ]:
@app.callback(
    Output('test-div', 'children'),
    [Input('slide-revenue', 'value')])
def callback_slide_value(slide_value):
    return 'You\'ve filtered by Revenue bigger than"{}" €'.format(str(slide_value))

HoverData

In [ ]:
@app.callback(
    [Output('hover-data', 'data'),
     Output('hover-data', 'columns')],
    [Input('map-companies', 'clickData')]
     )
def callback_image(hoverData):
    lon=hoverData['points'][0]['lon']
    lat=hoverData['points'][0]['lat']
    
    #Filter by lon and lat
    filtered_df = df_join[(df_join['Coordenada - X'] == lon) & (df_join['Coordenada - Y'] == lat)]          

    #filtered_df = df_join.iloc[(df_join['Coordenada - X']-lon).abs().argsort()[:1]] 
    #filtered_df = filtered_df.iloc[(filtered_df['Coordenada - Y']-lat).abs().argsort()[:1]] 
    
    columns = [{'name': col, 'id': col} for col in filtered_df.columns]
    data = filtered_df.to_dict(orient='records')
    return data,columns

In [ ]:
@app.callback(Output('map-companies', 'figure'),
              [Input('update-button', 'n_clicks')],
              [State('type-dropdown', 'value'),
              State('year-dropdown', 'value'),
              State('slide-revenue', 'value') ])

def update_graph(n_clicks, company_type, year_selected, revenue_slide):
    #Filter by CNAE type
    filtered_df = df_join[df_join['CNAE2009'].isin(company_type)]
    #Filter by Year
    filtered_df = filtered_df[(filtered_df['Año'] == int(year_selected))]  
    #Filter by Revenue
    filtered_df = filtered_df[(filtered_df['Ingresos de explotación'] > int(revenue_slide))] 

    
    #Updating the data to be shown
    data = [go.Scattermapbox(
                visible = True,
                lon = filtered_df['Coordenada - X'],
                lat = filtered_df['Coordenada - Y'],
                mode='markers',
                opacity=0.7,
                marker={'size': 15},
                name='Name_Prueba',
                text=[
                    filtered_df['Nombre'].values[i] + '<br>' + \
                                    'Ingresos de explotación: ' +\
                    str(filtered_df['Ingresos de explotación'].values[i]) + ' €' + '<br>' + \
                                    'Número de empleados: ' +\
                    str(filtered_df['Número empleados'].values[i]) \
                    for i in range(filtered_df.shape[0])
                ]
        

            )
    ]
    layout = go.Layout(
                    title="Brujula ingenieros Updated",
                    colorway=["#5E0DAC", '#FF4F00', '#375CB1', '#FF7400', '#FFF400', '#FF0056'],
                    template='plotly_dark',
                    paper_bgcolor='rgba(0, 0, 0, 0)',
                    plot_bgcolor='rgba(0, 0, 0, 0)',
                    margin={'t': 50},
                    height=800,
                    hovermode='closest',
                    mapbox=dict(accesstoken=mapbox_access_token,
                                        bearing=0,
                                        center=go.layout.mapbox.Center(
                                                lat=40.2085,
                                                lon=-3.7130),
                                        pitch=0,
                                        zoom=5)
    )
    
    #fig = go.Figure(
    #        px.scatter_mapbox(
    #            data_frame=filtered_df, 
    #            lat='Coordenada - Y', 
     #           lon='Coordenada - X',
      #          mapbox_style = 'outdoors',
       #         opacity=0.5, 
        #        zoom=4
         #   )
    #)
    
    
    fig = go.Figure(data=data, layout=layout)
    
   
    return fig

In [ ]:
if __name__ == '__main__':
    app.run_server()
